In [ ]:
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Check shapes
print(f'Train shape: {x_train.shape}, Test shape: {x_test.shape}')

import numpy as np
def create_dummy_boxes(y_labels, num_images):
    boxes = []
    for i in range(num_images):
        # Create a dummy bounding box: [x_min, y_min, x_max, y_max]
        box = np.array([10, 10, 22, 22])  # Example box coordinates
        boxes.append(box)
    return np.array(boxes)

# Create dummy boxes for training and testing
dummy_boxes_train = create_dummy_boxes(y_train, len(x_train))
dummy_boxes_test = create_dummy_boxes(y_test, len(x_test))

from tensorflow.keras import layers, models

def create_simple_yolo_model(input_shape=(32, 32, 3)):
    model = models.Sequential()

    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2)))

    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dense(4))  # Output: [x_min, y_min, x_max, y_max]

    model.compile(optimizer='adam', loss='mean_squared_error')

    return model

# Create the model
yolo_model = create_simple_yolo_model()

yolo_model.fit(x_train, dummy_boxes_train, epochs=5, batch_size=32, validation_data=(x_test, dummy_boxes_test))

import matplotlib.pyplot as plt
# Make predictions
predictions = yolo_model.predict(x_test)

# Display predictions with corresponding images
def visualize_predictions(images, true_boxes, pred_boxes, num_images=5):
    for i in range(num_images):
        plt.imshow(images[i])

        # True boxes
        true_box = true_boxes[i]
        plt.gca().add_patch(plt.Rectangle((true_box[0], true_box[1]),
                                            true_box[2] - true_box[0],
                                            true_box[3] - true_box[1],
                                            edgecolor='blue', facecolor='none', linewidth=2, label='True Box'))

        # Predicted boxes
        pred_box = pred_boxes[i]
        plt.gca().add_patch(plt.Rectangle((pred_box[0], pred_box[1]),
                                            pred_box[2] - pred_box[0],
                                            pred_box[3] - pred_box[1],
                                            edgecolor='red', facecolor='none', linewidth=2, label='Pred Box'))

        plt.title(f'Image {i + 1}')
        plt.axis('off')
        plt.legend(loc='upper right')
        plt.show()

# Visualize predictions
visualize_predictions(x_test, dummy_boxes_test, predictions)



170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Train shape: (50000, 32, 32, 3), Test shape: (10000, 32, 32, 3)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
 304/1563 ━━━━━━━━━━━━━━━━━━━━ 1:46 85ms/step - loss: 29.9150